In [16]:
## The only packages we would need for this pipeline are `Requests`, `os`, `Pandas`, `SQLite`, and `Streamlit`.
## Requests is to make HTTP requests to fetch the data from REST API endpoints
## And Pandas is for data transformations and wrangling.

import requests
import pandas as pd
import os

key = os.getenv('API_KEY')

response = requests.get("https://jsonplaceholder.typicode.com/users").json()

columns=['customer_id',
        'name',
        'username',
        'email',
        'lat',
        'lng']

users = pd.DataFrame(columns=columns)

i = 0

for rec in response:
    users.loc[i] = ([rec['id'],
                     rec['name'],
                     rec['username'], 
                     rec['email'],
                     rec['address']['geo']['lat'],
                     rec['address']['geo']['lng']])
    i += 1

sales = pd.read_csv('sales_data.csv').merge(users, on='customer_id')
sales



,order_id,customer_id,product_id,quantity,price,order_date,name,username,email,lat,lng
0,2334,5,40,3,35.60,2022-06-21,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,-31.8129,62.5342
1,6588,5,26,1,15.87,2022-10-23,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,-31.8129,62.5342
2,3569,5,47,9,19.35,2023-05-25,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,-31.8129,62.5342
3,5200,5,50,6,82.12,2023-02-28,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,-31.8129,62.5342
4,1589,5,10,1,96.94,2023-01-05,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,-31.8129,62.5342
...,...,...,...,...,...,...,...,...,...,...,...
995,9985,6,2,10,24.47,2023-02-21,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,-71.4197,71.7478
996,7971,6,11,9,82.94,2022-11-01,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,-71.4197,71.7478
997,9182,6,38,6,18.15,2023-04-22,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,-71.4197,71.7478
998,7967,6,34,10,14.44,2022-07-26,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,-71.4197,71.7478
